### Basic EDA

In [ ]:
# !pip install -Uq koreanize-matplotlib wordcloud

In [ ]:
import re
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import koreanize_matplotlib
import matplotlib.pyplot as plt

conn = sqlite3.connect('yes24_books_it.db')
df = pd.read_sql_query("SELECT * FROM books", conn)
conn.close()

In [ ]:
df

## 기본 EDA

In [ ]:
df.info()

In [ ]:
df["pub_year"] = df["pub_date"].map(lambda x : int(x.split("년")[0]))
df["pub_month"] = df["pub_date"].map(lambda x : int(x.split(" ")[-1].replace("월", "")))

In [ ]:
df["price_int"] = df["price"].str.replace(",", "").astype(int)
# 문자열을 float으로 변환
df["review_count_int"] = df["review_count"].replace(["N/A", ""], np.nan).astype(float)
df["rating_int"] = df["rating"].replace(["N/A", ""], np.nan).astype(float)

In [ ]:
df.describe().round(0)

In [ ]:
df.hist(bins=50);

### 파생변수 생성

In [ ]:
df["document"] = df["title"] + " " + df["description"]
df.head()

### 워드클라우드 시각화 

In [ ]:
# colab에 필수 라이브러리 설치
# !apt -qq -y install fonts-nanum

# 나눔 폰트 설치
import matplotlib.font_manager as fm
from wordcloud import WordCloud


# 모든 document를 하나의 문자열로 결합
text = " ".join(df["document"])

# 워드클라우드 생성
wordcloud = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', 
                      background_color='white').generate(text)

# 워드클라우드 시각화
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

### 출판사별 빈도

In [ ]:
# 빈도 계산
publisher_counts = df['publisher'].value_counts()

display(publisher_counts.head(10))

In [ ]:
publisher_counts.head(30).sort_values().plot(kind="barh")

In [ ]:
# 워드클라우드 생성
wordcloud = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', background_color='black').generate_from_frequencies(publisher_counts)

# 워드클라우드 시각화
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

### 정규표현식 적용

1. **숫자 추출**: `\d+` (텍스트에서 숫자(예: 가격, 날짜 등)를 추출)

2. **한글 단어 추출**: `[가-힣]+` (텍스트에서 한글 단어를 추출)

3. **URL 검출**: `https?://\S+` (텍스트에서 URL을 검출)

4. **특수문자 제거**: `[^가-힣a-zA-Z0-9\s]` (텍스트에서 특수문자를 제거)

5. **특정 패턴의 문자열 검출** (예: '챗GPT' 단어): `챗GPT` (텍스트에서 '챗GPT' 단어 검출)

6. **영어 단어 추출**: `[a-zA-Z]+` (텍스트에서 영어 단어를 추출)

7. **공백 제거**: `\s+` (텍스트에서 모든 공백을 제거)

8. **이메일 주소 검출**: `[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}` (텍스트에서 이메일 주소를 검출)

9. **전화번호 검출**: `\d{2,3}-\d{3,4}-\d{4}` (텍스트에서 전화번호를 검출)

10. **문장 끝 마침표 검출**: `\.\s` (텍스트에서 문장 끝의 마침표를 검출)

11. **이메일 마스킹**: `(?<=.{3}).(?=.*@)` (텍스트에서 이메일 주소의 일부를 마스킹 처리)

12. **전화번호 마스킹**: `(?<=\d{2,3}-)\d{3,4}(?=-\d{4})` (텍스트에서 전화번호의 중간 부분을 마스킹 처리)

13. **주민등록번호 마스킹**: `(?<=\d{6}-)\d{7}` (텍스트에서 주민등록번호의 뒷자리를 마스킹 처리)

14. **신용카드 번호 마스킹**: `(?<=\d{4}-)\d{4}-(?=\d{4}-\d{4})` (텍스트에서 신용카드 번호의 중간 부분을 마스킹 처리)

15. **비밀번호 마스킹**: `(?<=.{2}).` (텍스트에서 비밀번호의 일부를 마스킹 처리)

In [ ]:
# 앞뒤가 공백이고 글자수가 하나인 텍스트 제거 함수
def remove_single_char_words(text):
    return re.sub(r'\s.\s', ' ', text)

# 함수 적용
df['cleaned_document'] = df['document'].apply(remove_single_char_words)

# 모든 document를 하나의 문자열로 결합
text = " ".join(df["document"])
text[:1000]

### 불용어 처리 

In [ ]:
# 한국어 불용어 리스트
stopwords = set(["의", "이", "가", "은", "는", "에", "와", "과", "수",
                 "도", "을", "를", "으로", "로", "에게", "에서", "있는",
                 "책은", "및", "까지", "부터", "그리고", "하다", "있다", "있도록", "입니다"])

# 워드클라우드 생성
wordcloud = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', 
                      background_color='white', 
                      stopwords=stopwords).generate(text)

# 워드클라우드 시각화
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()